# unified_with_date

In [1]:
from nltk.tokenize import MWETokenizer, TweetTokenizer, WordPunctTokenizer 
from nltk.tokenize import word_tokenize
import re

VALUES = ['>>>=', '>>=', '<<=',  '%=', '^=', '|=', '&=', '/=',
                  '*=', '-=', '+=', '<<', '--', '++', '||', '&&', '!=',
                  '>=', '<=', '==', '%', '^', '|', '&', '/', '*', '-',
                  '+', ':', '?', '~', '!', '<', '>', '=', '...', '->', '::', '\\', '\\\\', '\*', '*\\', '\\\\\\']
INFIX = ['||', '&&', '|', '^', '&', '==', '!=', '<', '>', '<=', '>=', 
             '<<', '>>', '>>>', '+', '-', '*', '/', '%']
PREFIX = ['++', '--', '!', '~', '+', '-']
POSTFIX = ['++', '--']
ASSIGNMENT = ['=', '+=', '-=', '*=', '/=', '&=', '|=', '^=', '%=', '<<=', '>>=', '>>>=']
LAMBDA = ['->']
COMMENT = ['//', '/*', '*/']
METHOD_REFERENCE = ['::']

WHITESPACE_DICT = {"                ":'<|16-s|>', 
                   "            ":'<|12-s|>', 
                   "        ":'<|8-s|>', 
                   "    ":'<|4-s|>', 
                   "  ":'<|2-s|>', " ":'<|s|>',
                   "\t\t\t\t":'<|4-t|>',"\t\t\t":'<|3-t|>',"\t\t":'<|2-t|>',"\t":'<|t|>',"\n":'<|nl|>'}

WHITESPACES = [WHITESPACE_DICT[x] for x in WHITESPACE_DICT]

REVERSE_WHITESPACE_DICT = {}
for key in WHITESPACE_DICT:
    value = WHITESPACE_DICT[key]
    REVERSE_WHITESPACE_DICT[value] = key

CUSTOM_TOKEN = ["<|startcode|>", "<|endcode|>", "<|startfocus|>", "<|endfocus|>", "<|startcomment|>", "<|endcomment|>", 
                   "<|stringliteral|>", "<|singlelinecomment|>", "<|multilinecomment|>", "<|del|>"]

values = list(set(INFIX+PREFIX+POSTFIX+ASSIGNMENT+LAMBDA+COMMENT+METHOD_REFERENCE))
token_phrases = []
word_punct_tokenizer = WordPunctTokenizer()
for tok in CUSTOM_TOKEN+WHITESPACES:
    temp = tuple(word_punct_tokenizer.tokenize(tok))
    token_phrases.append(temp)
for w in values:
    temp = tuple(MWETokenizer().tokenize(w))
    if len(temp) > 1:
        token_phrases.append(temp)

word_punct_tokenizer = WordPunctTokenizer()
tweet_tokenizer = TweetTokenizer()
mwe_tokenizer = MWETokenizer(token_phrases, separator="")

def state(c):
    n = ord(c)
    if n>=97 and n<=122: # lower case
        return 1
    elif n>=65 and n<=90: # upper case
        return 2
    elif n>=48 and n<=57: # numbers
        return 3
    elif c.isspace(): # whitespaces
        return 4
    elif c in ['_', '$']: 
        return 5
    elif n < 128:
        return 6
    else:
        return 7

def space_up(s):
    if s is None or s == "":
        return ""
    new_s = s[0]
    for i in range(1,len(s)):
        prev_state = state(s[i-1])
        curr_state = state(s[i])
        if prev_state in [1,2] and curr_state in [3]:
            new_s += " "
        elif prev_state in [1] and curr_state in [2]:
            new_s += " "
        elif prev_state in [3] and curr_state in [1,2]:
            new_s += " "
        elif prev_state in [1,2,3] and curr_state in [5]:
            new_s += " "
        elif prev_state in [5] and curr_state in [1,2,3]:
            new_s += " "
        new_s+=s[i]
    return new_s

def white_space_tokenize(s):
    for x in WHITESPACE_DICT:
        s = s.replace(x, WHITESPACE_DICT[x])
    for key in REVERSE_WHITESPACE_DICT:
        #val = REVERSE_WHITESPACE_DICT[x]
        s = s.replace(key, " "+key+" ")
    return s

def extreme_tokenization(comment):
    comment = white_space_tokenize(comment)
    comment = space_up(comment)
    tokenized = tweet_tokenizer.tokenize(comment)
    tokenized = word_punct_tokenizer.tokenize(' '.join(tokenized))
    tokenized = mwe_tokenizer.tokenize(tokenized)
    tokenized_comment = ' '.join(tokenized)
    tokenized_comment = re.sub(r'[^\x00-\x7f]',r'', tokenized_comment)
    tokenized = tokenized_comment.split()
    return tokenized

def extreme_detokenization(tokens):
    s = ""
    for token in tokens:
        if token in REVERSE_WHITESPACE_DICT:
            s+= REVERSE_WHITESPACE_DICT[token]
        else:
            s+= token
    return s

In [60]:
import json
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data
def read(path):
    return open(path, 'r', encoding = 'utf-8').read()


BASE="E:/codes/"
unique_data = getJsonData('unique_data_processed_with_Date_multithread_idx_400_feb_03.json')
print(unique_data[0])

{'status': 'update', 'message': 'What\'s "us" ?', 'comment_id': 'f98e907f_b5dc8856', 'target': '     * <BR>Then the test sets a breakpoint so that the tested thread (DebuggeeThread) gets suspended\n     * by an event. Once this thread is suspended, we send it an ObjectReference.InvokeMethod\n     * command to initiate a method invocation executing in that thread. The method will synchronize\n     * with the test, waiting for a signal to continue its execution.\n     * <BR>While the tested thread waits for the signal, we send a VirtualMachine.Dispose command to\n     * the debuggee and sends the expected signal so the tested thread completes the method\n     * invocation.\n', 'code_snippet': "     * This testcase exercises VirtualMachine.Dispose command when a thread, suspended by an\n     * event, is still invoking a method.\n     * <BR>At first the test starts DisposeDuringInvokeDebuggee debuggee.\n<|startfocus|>\n     * <BR>Then the test sets a breakpoint a tested thread (DebuggeeThr

In [ ]:
from os import path
import json
import re
import subprocess
import nltk
from pprint import pprint
from nltk.tokenize import sent_tokenize, word_tokenize
from tqdm import tqdm
nltk.download('punkt')

def get_source_target(file1, file2, line_number, change_window_size=5):
    source_target = subprocess.check_output(['java', '-jar', 'ChangedLineF.jar', \
                                file1, file2, str(line_number), str(change_window_size)]).decode("utf-8")
    (source, target) = source_target.split("<|source-target-sep|>")
    return source, target

def end_scope(start_index, lines):
    #print(lines)
    counter = 0
    end_index = -1
    found = False
    for index in range(start_index, len(lines), 1):
        #print(lines[index])
        #print(counter)
        for char in lines[index]:
            if(char=='{'):
                counter+=1
                found = True
            elif(char == "}"):
                counter-=1
                found = True
            if(counter==0 and found):
                end_index = index
                break
        if end_index!=-1:
            break
    return end_index

unique_data_processed = []
MAX_NUM_TOKEN = 800
UP_TOKEN = MAX_NUM_TOKEN//2
c = 0
ec = 0
errors = []
tufano_less_then_50 = []
tufano_less_then_100 = []
vis_idx = []
#for idx, data in enumerate(unique_data):#[2428:2429]#[1000:1001]
for idx, data in enumerate(unique_data): #1:75
    #if(idx>3000):
    #    break
    if(idx%100==0):
        print(idx, len(tufano_less_then_50), len(tufano_less_then_100))
    #if len(tufano_less_then_50) >=10 and len(tufano_less_then_100)>=10:
    #    break
    if data['project_name']!='android_':
        continue
    code_file_name = data['code_file_name']
    base_patch_number = data['base_patch_number']
    changed_patch_number = data['changed_patch_number']
    main_code_path = BASE+code_file_name+'/'+str(base_patch_number)+'.java'
    changed_code_path = BASE+code_file_name+'/'+str(changed_patch_number)+'.java'
    line_no = data['base_code_line_number']
    main_code = read(main_code_path)
    changed_code = read(changed_code_path)
    sample = main_code
    message = data['message']
    comment_id = data['comment_id']
    #print("line no = ", line_no)
    sp_start_func= -1
    sp_end_func= len(sample.split('\n'))   
    funcs = re.findall(r"(public|protected|private|static|\s) +([\w\<\>\[\]]+\s+(\w+)) *\([^\)]*\) *(\{?|[^;])",sample)
    func_scopes = []
    lines = sample.split("\n")
    #print(funcs)
    #continue
    visited_func = set()
    for func in funcs:
        #print(func)
        if(func[-1]!= "{"):
            continue
        fc = list(func)
        fc = fc[-3]
        s = "".join(fc)
        s = s.strip()
        found = False
        for index in range(0, len(lines), 1):
            if s in lines[index]:
                start_func = index
                end_func = end_scope(index, lines)
                func_scopes.append((start_func,end_func))
                if s in visited_func:
                    continue
                if s not in visited_func:
                    visited_func.add(s)
                    #func_list.append(lines[index].replace("{", "").strip())
                if(start_func<=line_no<=end_func):
                    found = True
                    special_func = "\n".join(lines[start_func:end_func+1])
                    sp_start_func= start_func
                    sp_end_func= end_func
                    #print("comes")
                    if s not in extreme_detokenization(data['tokenized_code_snippet']):
                        continue
                    last_char =  extreme_detokenization(data['tokenized_code_snippet']).strip()[-1]
                    if last_char!='}':
                        continue
                    detokenized_code = extreme_detokenization(data['tokenized_code_snippet'])
                    #print(detokenized_code)
                    start_f = detokenized_code.find("<|startfocus|>\n")
                    end_f = detokenized_code.find("<|endfocus|>\n")
                    #print("se ", start_f, " ", end_f)
                    repl_st = detokenized_code[start_f:end_f+14]
                    #print(repl_st)
                    #print(data['target'])
                    with open("temp\\temp.java", 'w', encoding='utf-8') as f:
                        f.write(extreme_detokenization(data['tokenized_code_snippet']).replace("<|startfocus|>\n", "").replace("<|endfocus|>\n", ""))
                    with open("temp\\temp2.java", 'w', encoding='utf-8') as f:
                        f.write(extreme_detokenization(data['tokenized_code_snippet']).replace(repl_st, data['target'].replace("\n<|del|>\n", "")))#replace("\n<|del|>\n", ""))
                    
                    java_cmd = "java -jar src2abs-0.1-jar-with-dependencies.jar single method "
                    java_cmd = java_cmd + "temp\\temp.java"+" "
                    java_cmd = java_cmd+"temp\\processed.java"+' idioms.csv'
                    check = subprocess.call(java_cmd.replace('/', '\\'), shell=True)
                    #print(java_cmd)
                    if check != 0:
                        print(java_cmd)
                        print("java cmd execusion failed on file ", f)
                        continue
                    check = subprocess.call(java_cmd.replace('/', '\\').replace("temp.java", "temp2.java").replace("processed", "processed2"), shell=True)
                    #print(java_cmd)
                    if check != 0:
                        print(java_cmd)
                        print("java cmd execusion failed on file ", f)
                        continue
                    #print("comes")
                    file_data = read("temp\\processed.java")
                    file_data_y = read("temp\\processed2.java")
                    #print(file_data)
                    #print(idx," ",  len(file_data.split(" "))," ", len(file_data_y.split(" ")) )
                    if(len(file_data.split(" "))<=50 and len(file_data_y.split(" "))<=50):
                        dt = {}
                        dt['idx'] = idx
                        dt['status'] = data['status']
                        dt['global_index'] = data['global_index']
                        dt['project_name'] = data['project_name']
                        dt['written_on'] = data['written_on']
                        #dt['source'] = file_data
                        dt['tufano_target'] = file_data_y
                        dt['comment'] = data['message']
                        dt['tokenized_comment'] = data['tokenized_comment']
                        dt['tufano_code'] = file_data
                        dt['our_target'] = data['target']
                        dt['our_code'] = data['code_snippet'] #.replace("<|startfocus|>\n", "").replace("<|endfocus|>\n", "")
                        dt['tokenized_code'] = data['tokenized_code_snippet']
                        dt['written_on'] = data['written_on']
                        
                        dt['base_code_line_number'] = data['base_code_line_number']
                        dt['base_patch_number'] = data['base_patch_number']
                        dt['changed_patch_number'] = data['changed_patch_number']
                        dt['code_file_name'] = data['code_file_name']
                        dt['comment_id'] = data['comment_id']
                        dt['class_list'] = data['class_list']
                        dt['func_list'] = data['func_list']
                        dt['prime_var_dic'] = data['prime_var_dic']
                        #print("50->", idx)
                        tufano_less_then_50.append(dt)
                        #func_less_then_50.append(file_data)
                        break
                    if(len(file_data.split(" "))<=100 and len(file_data_y.split(" "))<=100):
                        dt = {}
                        dt['idx'] = idx
                        dt['status'] = data['status']
                        dt['global_index'] = data['global_index']
                        dt['project_name'] = data['project_name']
                        dt['written_on'] = data['written_on']
                        #dt['source'] = file_data
                        dt['tufano_target'] = file_data_y
                        dt['comment'] = data['message']
                        dt['tokenized_comment'] = data['tokenized_comment']
                        dt['tufano_code'] = file_data
                        dt['our_target'] = data['target']
                        dt['our_code'] = data['code_snippet'] #.replace("<|startfocus|>\n", "").replace("<|endfocus|>\n", "")
                        dt['tokenized_code'] = data['tokenized_code_snippet']
                        dt['written_on'] = data['written_on']
                        
                        dt['base_code_line_number'] = data['base_code_line_number']
                        dt['base_patch_number'] = data['base_patch_number']
                        dt['changed_patch_number'] = data['changed_patch_number']
                        dt['code_file_name'] = data['code_file_name']
                        dt['comment_id'] = data['comment_id']
                        dt['class_list'] = data['class_list']
                        dt['func_list'] = data['func_list']
                        dt['prime_var_dic'] = data['prime_var_dic']
                        tufano_less_then_100.append(dt)
                        break
                    

0 0 0


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


100 10 7
200 17 11
300 22 14
400 31 19
500 34 21
600 38 23
700 38 28
800 40 29
900 40 33
1000 43 37
1100 43 38
1200 44 42
1300 46 44
1400 50 45
1500 52 47
1600 54 48
1700 56 53
1800 57 54
1900 57 55
2000 57 56
2100 60 56
2200 60 56
2300 60 57
2400 61 59
2500 61 60
2600 61 62
2700 61 63
2800 62 64
2900 64 66
3000 64 69
3100 65 70
3200 67 71
3300 68 75
3400 68 76
3500 68 77
3600 68 78
3700 70 79
3800 71 82
3900 72 82
4000 74 82
4100 76 82
4200 76 82
4300 78 82
4400 79 83
4500 79 84
4600 81 85
4700 82 86
4800 86 88
4900 87 88
5000 88 90
5100 88 97
5200 89 99
5300 90 101
5400 91 102
5500 93 104
5600 95 106
5700 96 109
5800 96 111
5900 96 114
6000 96 115
6100 99 116
6200 101 116
6300 104 117
6400 107 117
6500 108 119
6600 110 119
6700 110 122
6800 112 123
6900 113 124
7000 113 125
7100 114 126
7200 117 126
7300 119 129
7400 122 132
7500 123 134
7600 126 137
7700 128 137
7800 131 138
7900 131 140
8000 131 143
8100 132 143
8200 133 144
8300 133 145
8400 133 146
8500 135 146
8600 136 146
8700 

In [ ]:
print(len(tufano_less_then_50))
print(len(tufano_less_then_100))
with open('less_then_50_on_400.json', 'w', encoding="utf8") as f:  # writing JSON object
    json.dump(tufano_less_then_50, f)
with open('less_then_100_on_400.json', 'w', encoding="utf8") as f:  # writing JSON object
    json.dump(tufano_less_then_100, f)

In [72]:
pprint(tufano_less_then_50)

[{'base_code_line_number': 71,
  'base_patch_number': 11,
  'changed_patch_number': 12,
  'code_file_name': 'android_4548',
  'comment': 'missing space before the new.',
  'comment_id': 'd06988a3_4c321266',
  'global_index': 13001,
  'idx': 12,
  'our_code': '  public Iterable<ModuleTemplate> getModuleTemplates() {\n'
              '<|startfocus|>\n'
              '    myModuleTemplates = Lists.newArrayList((ModuleTemplate)new '
              'NewCloudModuleTemplate(myTemplates));\n'
              '<|endfocus|>\n'
              '    return myModuleTemplates;\n'
              '  }',
  'our_target': '    myModuleTemplates = '
                'Lists.newArrayList((ModuleTemplate)new '
                'NewCloudModuleTemplate());\n',
  'project_name': 'android_',
  'tokenized_code': ['<|2-s|>',
                     'public',
                     '<|s|>',
                     'Iterable',
                     '<',
                     'Module',
                     'Template',
                

                     '<|nl|>',
                     '<|4-s|>',
                     '}'],
  'tokenized_comment': ['Similarly',
                        ',',
                        '<|s|>',
                        'this',
                        '<|s|>',
                        'seems',
                        '<|s|>',
                        'to',
                        '<|s|>',
                        'belong',
                        '<|s|>',
                        'in',
                        '<|s|>',
                        'Keymaster',
                        'Arguments',
                        '.'],
  'tufano_code': 'public static void METHOD_1 ( TYPE_1 args , int VAR_1 , int '
                 '... values ) { for ( int value : values ) { args . METHOD_2 '
                 '( VAR_1 , value ) ; } }',
  'tufano_target': 'public static void METHOD_1 ( TYPE_1 args , int VAR_1 , '
                   'int ... values ) { for ( int value : values ) { args . '
                   'METH

                     '<|startfocus|>',
                     '<|nl|>',
                     '<|4-s|>',
                     'synchronized',
                     '<|s|>',
                     '(',
                     'all',
                     'Users',
                     ')',
                     '<|s|>',
                     '{',
                     '<|nl|>',
                     '<|endfocus|>',
                     '<|nl|>',
                     '<|4-s|>',
                     '<|2-s|>',
                     'all',
                     'Users',
                     '.',
                     'clear',
                     '(',
                     ');',
                     '<|nl|>',
                     '<|4-s|>',
                     '<|2-s|>',
                     'all',
                     'Users',
                     '.',
                     'put',
                     'All',
                     '(',
                     'users',
                     ');',
                 

  'tufano_target': 'private static TYPE_1 METHOD_1 ( TYPE_1 VAR_1 ) { int '
                   'VAR_2 = 0 ; while ( VAR_2 ++ < INT_1 ) { if ( VAR_1 . '
                   'METHOD_2 ( ) == null ) return VAR_1 ; VAR_1 = VAR_1 . '
                   'METHOD_2 ( ) ; } return VAR_1 ; }',
  'written_on': '2015-08-04 19:24:31'},
 {'base_code_line_number': 374,
  'base_patch_number': 7,
  'changed_patch_number': 8,
  'code_file_name': 'android_3405',
  'comment': 'technology,etc.',
  'comment_id': '6a31fb81_3c5f1954',
  'global_index': 3007,
  'idx': 133,
  'our_code': '<|startfocus|>\n'
              '    private void notifyHostEmuActivated(int type) {\n'
              '        mListener.onHostCardEmulationActivated(type);\n'
              '<|endfocus|>\n'
              '    }',
  'our_target': '    private void notifyHostEmuActivated(int technology) {\n'
                '        mListener.onHostCardEmulationActivated(technology);\n',
  'project_name': 'android_',
  'tokenized_code': ['<|star

                     '<|s|>',
                     '{',
                     '<|nl|>',
                     '<|8-s|>',
                     '//',
                     '@',
                     'uses',
                     '<|s|>',
                     'dot',
                     '.',
                     'junit',
                     '.',
                     'opcodes',
                     '.',
                     'filled',
                     '_',
                     'new',
                     '_',
                     'array',
                     '_',
                     'range',
                     '.',
                     'd',
                     '.',
                     'T',
                     '_',
                     'filled',
                     '_',
                     'new',
                     '_',
                     'array',
                     '_',
                     'range',
                     '_',
                     '11',
                     '<|

In [ ]:
tufano_less_then_50 = getJsonData("E:\APR\DATA\less_then_50_on_400.json")
tufano_less_then_100 = getJsonData("E:\APR\DATA\less_then_50_on_400.json")

# Remove Duplicates

In [ ]:
from os import path
import json
import re
import subprocess
import nltk
from pprint import pprint
from nltk.tokenize import sent_tokenize, word_tokenize
from tqdm import tqdm
nltk.download('punkt')

def merge_code_comment(x):
    message = x['message']
    source = ' '.join(x['tokenized_code_snippet'])
    target = ' '.join(x['target'])
    merged = message + '<>' + source + '<>' + target
    return merged

def get_source_target(file1, file2, line_number, change_window_size=5):
    source_target = subprocess.check_output(['java', '-jar', 'ChangedLineF.jar', \
                                file1, file2, str(line_number), str(change_window_size)]).decode("utf-8")
    (source, target) = source_target.split("<|source-target-sep|>")
    return source, target

def end_scope(start_index, lines):
    #print(lines)
    counter = 0
    end_index = -1
    found = False
    for index in range(start_index, len(lines), 1):
        #print(lines[index])
        #print(counter)
        for char in lines[index]:
            if(char=='{'):
                counter+=1
                found = True
            elif(char == "}"):
                counter-=1
                found = True
            if(counter==0 and found):
                end_index = index
                break
        if end_index!=-1:
            break
    return end_index

import traceback
existing_ids = set()
duplicate_ids = set()
unique_dat = []
i = 0
target_size = 400
error_count = 0
error_list = []
'''
for dt in tufano_less_then_50:
    #print(dt['idx'])
    x = unique_data[dt['idx']]
    i+= 1
    try:
        focuslen = x['tokenized_code_snippet'].index('<|endfocus|>') - x['tokenized_code_snippet'].index('<|startfocus|>')
        key = merge_code_comment(x)
        if key not in existing_ids and len(x['tokenized_target']) <= target_size and focuslen <= target_size:
            existing_ids.add(key)
            unique_dat.append(i-1)
        elif key not in duplicate_ids:
            duplicate_ids.add(key)
    except Exception as e:
        print(e)
        x['error'] = e
        error_list.append(x)
        error_count += 1
        #print(x)
        #break
print(len(unique_dat))
'''

for dt in tufano_less_then_100:
    #print(dt['idx'])
    x = unique_data[dt['idx']]
    i+= 1
    try:
        focuslen = x['tokenized_code_snippet'].index('<|endfocus|>') - x['tokenized_code_snippet'].index('<|startfocus|>')
        key = merge_code_comment(x)
        if key not in existing_ids and len(x['tokenized_target']) <= target_size and focuslen <= target_size:
            existing_ids.add(key)
            unique_dat.append(i-1)
        elif key not in duplicate_ids:
            duplicate_ids.add(key)
    except Exception as e:
        print(e)
        x['error'] = e
        error_list.append(x)
        error_count += 1
        #print(x)
        #break
print(len(unique_dat))
'''
manual_less_then_50 = []
manual_less_then_100 = []
for i in unique_dat: #1:75
    mnl = {}
    dt = tufano_less_then_100[i]
    #print(dt['idx'])
    #print(dt['source'])
    data = unique_data[dt['idx']]
    code_file_name = data['code_file_name']
    base_patch_number = data['base_patch_number']
    changed_patch_number = data['changed_patch_number']
    main_code_path = BASE+code_file_name+'/'+str(base_patch_number)+'.java'
    changed_code_path = BASE+code_file_name+'/'+str(changed_patch_number)+'.java'
    line_no = data['base_code_line_number']
    main_code = read(main_code_path)
    changed_code = read(changed_code_path)
    sample = main_code
    message = data['message']
    comment_id = data['comment_id']
    detokenized_code = extreme_detokenization(data['tokenized_code_snippet'])
    #print(detokenized_code.replace("<|startfocus|>\n", "").replace("<|endfocus|>\n", ""))
    mnl['idx'] = dt['idx']
    mnl['comment'] = message
    mnl['tokenized_comment'] = data['tokenized_comment']
    mnl['tufano_code'] = dt['source']
    mnl['our_code'] = detokenized_code#.replace("<|startfocus|>\n", "").replace("<|endfocus|>\n", "")
    mnl['tokenized_code'] = data['tokenized_code_snippet']
    mnl['written_on'] = dt['written_on']
    
    start_f = detokenized_code.find("\n<|startfocus|>\n")
    end_f = detokenized_code.find("\n<|endfocus|>\n")
    repl_st = detokenized_code[start_f:end_f+15]
    #print(repl_st)
    mnl['our_target'] = target
    manual_less_then_100.append(mnl)
#with open('manual_less_then_100.json', 'w', encoding="utf8") as f:  # writing JSON object
#    json.dump(manual_less_then_100, f)
print(len(manual_less_then_100))
#manual_less_then_100[0]
'''

In [ ]:
#print(unique_data[0].keys())
print(tufano_less_then_50[0].keys())

In [ ]:
import json
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data
def read(path):
    return open(path, 'r', encoding = 'utf-8').read()


BASE="C:/research_stuff/codes/"
unique_data = getJsonData('E:\\unified_with_date.json')
print(len(unique_data))

In [ ]:
pprint(unique_data[768])

In [27]:
dt = getJsonData('manual_less_then_50.json')
print(len(dt))
print(dt[0]['idx'])
for key in dt[0]:
    if(dt[0][key]!=tufano_less_then_50[0][key]):
        print(dt[0][key])
        print(tufano_less_then_50[0][key])
        print(dt[0]['our_code'])

#print(dt[0]['idx'])
#tufano_less_then_50[0]['idx']

912
37
private static void VAR_1 ( ) { < | VAR_2 | > for ( int i = 0 ; i < < | VAR_3 | > VAR_4 ; i ++ ) { < | VAR_5 | > new VAR_6 ( false ) ; } }
private static void VAR_1 ( ) { for ( int i = 0 ; i < < | VAR_2 | > VAR_3 ; i ++ ) { new VAR_4 ( false ) ; } }
	private static void allocateUnreachableObjects() {
<|startfocus|>
        for(int i = 0; i <<|s|>unreachableObjNum; i++) {
<|endfocus|>
            new MockClass(false);
        }
	}


In [25]:
tufano_less_then_50[0].keys()

dict_keys(['idx', 'project_name', 'written_on', 'tufano_target', 'comment', 'tokenized_comment', 'tufano_code', 'our_target', 'our_code', 'tokenized_code'])

In [58]:
data = unique_data[37]
print(extreme_detokenization(extreme_tokenization(data['source'][2020:2200])))

print(extreme_detokenization(data['tokenized_code_snippet']))

private static void allocateUnreachableObjects() {
<|startfocus|>
        for(int i = 0; i < unreachableObjNum; i++) {
<|endfocus|>
            new MockClass(false);
        }
	}


	private static void allocateUnreachableObjects() {
<|startfocus|>
        for(int i = 0; i <<|s|>unreachableObjNum; i++) {
<|endfocus|>
            new MockClass(false);
        }
	}
